feature_engineering.ipynb 를 일부 수정해서 만들었습니다.

In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import defaultdict

## Raw data load

In [2]:
raw_data = pd.read_csv('/opt/ml/input/data/FE_total_data.csv')
raw_data.sort_values(by=["userID", "Timestamp"], inplace=True)
df = raw_data.copy()

### recCount : 최근 누적 정답 갯수

In [3]:
raw_data['recAccuracy'] = 0

group = raw_data.groupby("userID")
group.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,recAccuracy
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,0
...,...,...,...,...,...,...,...,...
2526691,7441,A030071001,A030000071,0,2020-06-05 06:46:41,438,1,0
2526692,7441,A030071002,A030000071,0,2020-06-05 06:48:43,438,1,0
2526693,7441,A030071003,A030000071,1,2020-06-05 06:49:23,438,1,0
2526694,7441,A030071004,A030000071,0,2020-06-05 06:49:57,438,1,0


In [4]:
def set_rec_count(group):
    group.sort_values(by='Timestamp', axis=0, inplace=True)
    recAccuracy = group['answerCode'].rolling(10, min_periods=1).sum().shift(1)
    group['recCount'] = recAccuracy
    
    return group

In [7]:
raw_data['recCount'] = 0

group = raw_data.groupby("userID")

group = group.apply(set_rec_count)
group.reset_index(drop=True, inplace=True)

raw_data['recCount'] = group['recCount']
raw_data['recCount']=raw_data['recCount'].fillna(0)
raw_data['recCount'] = raw_data['recCount'].astype(int)
raw_data.head(10)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,recAccuracy,recCount
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,0,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,0,2
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,0,3
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,0,4
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,1,0,5
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,1,0,6
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,1,0,6
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,1,0,7
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,1,0,8


In [8]:
raw_data.to_csv('/opt/ml/input/data/FE/recCount.csv')